In [15]:
!pip install python-bcb pandas seaborn sklearn yfinance

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [16]:
import pandas as pd
import seaborn as sns
import yfinance as yf
from bcb import sgs
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.preprocessing import MinMaxScaler

In [17]:
tickers = ['^GSPC', '^VIX', '^TNX',  'BZ=F', 'BRL=X', 'IRFM11.SA']
dataframes = []
closes = []
for ticker in tickers:
    dado = yf.download(
        ticker,
        start='2020-01-01',
        end='2025-08-01',
        auto_adjust=True
    )

    if not dado.empty:
        close = dado[['Close']].rename(columns={'Close': ticker})
        closes.append(close)
df_final = pd.concat(closes, axis=1)
print(df_final)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price             ^GSPC       ^VIX   ^TNX       BZ=F     BRL=X  IRFM11.SA
Ticker            ^GSPC       ^VIX   ^TNX       BZ=F     BRL=X  IRFM11.SA
Date                                                                     
2020-01-01          NaN        NaN    NaN        NaN  4.016786        NaN
2020-01-02  3257.850098  12.470000  1.882  66.250000  4.016300  61.880001
2020-01-03  3234.850098  14.020000  1.788  68.599998  4.023400  61.959999
2020-01-06  3246.280029  13.850000  1.811  68.910004  4.057000  61.950001
2020-01-07  3237.179932  13.790000  1.827  68.269997  4.060400  61.939999
...                 ...        ...    ...        ...       ...        ...
2025-07-25  6388.640137  14.930000  4.386  68.440002  5.520000  90.989998
2025-07-28  6389.770020  15.030000  4.420  70.040001  5.565200  91.070000
2025-07-29  6370.859863  15.980000  4.330  72.510002  5.586800  91.279999
2025-07-30  6362.899902  15.480000  4.376  73.239998  5.573200  91.330002
2025-07-31  6339.390137  16.719999  4.

In [18]:
inicio = '2020-01-01'
fim = '2025-08-01'

ipca_exp = sgs.get(433, start=inicio, end=fim)
ipca_exp.rename(columns={433: 'IPCA_Expectativa_12m'}, inplace=True)

selic = sgs.get(432, start=inicio, end=fim)
selic.rename(columns={432: 'Selic_Meta'}, inplace=True)

ibcbr = sgs.get(24363, start=inicio, end=fim)
ibcbr.rename(columns={24363: 'IBC_Br'}, inplace=True)

ptax = sgs.get(1, start=inicio, end=fim)
ptax.rename(columns={1: 'Dolar_PTAX'}, inplace=True)

macro_bcb = {
    'IPCA_EXP': ipca_exp,
    'SELIC': selic,
    'IBC_BR': ibcbr,
    'PTAX': ptax
}
macro_bcb['SELIC'].describe()

macro_df = pd.concat(macro_bcb.values(), axis=1)
macro_df.head()
macro_df = macro_df.ffill()
macro_df.head()

,433,432,24363,1
Date,,,,
2020-01-01,0.21,4.5,94.02696,NaN
2020-01-02,0.21,4.5,94.02696,4.0213
2020-01-03,0.21,4.5,94.02696,4.0522
2020-01-04,0.21,4.5,94.02696,4.0522
2020-01-05,0.21,4.5,94.02696,4.0522


In [19]:
join = [macro_df, df_final]

externo = pd.concat(join, axis=1)
print(externo)

             433   432      24363       1  (^GSPC, ^GSPC)  (^VIX, ^VIX)  \
Date                                                                      
2020-01-01  0.21   4.5   94.02696     NaN             NaN           NaN   
2020-01-02  0.21   4.5   94.02696  4.0213     3257.850098     12.470000   
2020-01-03  0.21   4.5   94.02696  4.0522     3234.850098     14.020000   
2020-01-04  0.21   4.5   94.02696  4.0522             NaN           NaN   
2020-01-05  0.21   4.5   94.02696  4.0522             NaN           NaN   
...          ...   ...        ...     ...             ...           ...   
2025-07-28  0.26  15.0  113.16809  5.5877     6389.770020     15.030000   
2025-07-29  0.26  15.0  113.16809  5.5764     6370.859863     15.980000   
2025-07-30  0.26  15.0  113.16809  5.6034     6362.899902     15.480000   
2025-07-31  0.26  15.0  113.16809  5.6021     6339.390137     16.719999   
2025-08-01 -0.11  15.0  110.42518  5.5436             NaN           NaN   

            (^TNX, ^TNX)

In [20]:
url_focus = (
    "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/"
    "ExpectativasMercadoAnuais?$top=100000"
    "&$filter=Indicador%20eq%20'IPCA'%20and%20Data%20ge%20'2020-01-01'"
    "&$format=json"
    "&$orderby=Data%20asc"
)
try:
    json = pd.read_json(url_focus)
    Focus = pd.DataFrame(json['value'].tolist())

    Focus['Data'] = pd.to_datetime(Focus['Data'])

    Focus['DataReferencia'] = Focus['DataReferencia'].astype(int)
    
    Focus['Ano_Divulgacao'] = Focus['Data'].dt.year
    
    Focus = Focus[Focus['DataReferencia'] == Focus['Ano_Divulgacao']].copy()

    Focus = Focus.set_index('Data').sort_index()
    
    Focus = Focus[['Mediana']].rename(columns={'Mediana': 'IPCA_Expectativa_AnoCorrente'})
    
    print(Focus.tail())

except Exception as e:
    raise TypeError
print(Focus)

            IPCA_Expectativa_AnoCorrente
Data                                    
2025-12-10                        4.3579
2025-12-11                        4.3683
2025-12-11                        4.3579
2025-12-12                        4.3579
2025-12-12                        4.3540
            IPCA_Expectativa_AnoCorrente
Data                                    
2020-01-02                        3.6000
2020-01-02                        3.5800
2020-01-03                        3.6000
2020-01-03                        3.5900
2020-01-06                        3.6000
...                                  ...
2025-12-10                        4.3579
2025-12-11                        4.3683
2025-12-11                        4.3579
2025-12-12                        4.3579
2025-12-12                        4.3540

[2990 rows x 1 columns]


In [21]:
Focus = Focus.groupby(Focus.index).mean()
Focus = Focus.sort_index()
print(Focus)

            IPCA_Expectativa_AnoCorrente
Data                                    
2020-01-02                       3.59000
2020-01-03                       3.59500
2020-01-06                       3.58000
2020-01-07                       3.58000
2020-01-08                       3.58500
...                                  ...
2025-12-08                       4.39420
2025-12-09                       4.38960
2025-12-10                       4.36310
2025-12-11                       4.36310
2025-12-12                       4.35595

[1495 rows x 1 columns]


In [22]:
FeatureBase = pd.merge(externo, Focus, left_index=True, right_index=True, how='inner')
FeatureSemanal = FeatureBase[FeatureBase.index.dayofweek == 4].copy()
print(FeatureSemanal)

             433   432      24363       1  (^GSPC, ^GSPC)  (^VIX, ^VIX)  \
2020-01-03  0.21   4.5   94.02696  4.0522     3234.850098         14.02   
2020-01-10  0.21   4.5   94.02696  4.0745     3265.350098         12.56   
2020-01-17  0.21   4.5   94.02696  4.1837     3329.620117         12.10   
2020-01-24  0.21   4.5   94.02696  4.1769     3295.469971         14.56   
2020-01-31  0.21   4.5   94.02696  4.2695     3225.520020         18.84   
...          ...   ...        ...     ...             ...           ...   
2025-07-04  0.26  15.0  113.16809  5.4090             NaN           NaN   
2025-07-11  0.26  15.0  113.16809  5.5722     6259.750000         16.40   
2025-07-18  0.26  15.0  113.16809  5.5466     6296.790039         16.41   
2025-07-25  0.26  15.0  113.16809  5.5426     6388.640137         14.93   
2025-08-01 -0.11  15.0  110.42518  5.5436             NaN           NaN   

            (^TNX, ^TNX)  (BZ=F, BZ=F)  (BRL=X, BRL=X)  \
2020-01-03         1.788     68.599998   

In [30]:
finbert = pd.read_csv('FinBert.csv')
finbert['data'] = pd.to_datetime(finbert['data'])

Diario = finbert.groupby('data')['score'].sum().sort_index()
Diario = Diario.to_frame(name='scoreTotal')

Diario['scoreSemanal'] = Diario['scoreTotal'].rolling('7D', min_periods=1).mean()

FeatureB = Diario.loc[Diario.index.dayofweek == 4, ['scoreSemanal']].copy()

print(FeatureB.tail())

            scoreSemanal
data                    
2025-09-26      4.593504
2025-10-03      3.399397
2025-10-10      3.692898
2025-10-17      3.071258
2025-10-24      3.468857


In [31]:
Features_B = pd.merge(FeatureB, FeatureSemanal, left_index=True, right_index=True, how='inner')
print(Features_B)

            scoreSemanal   433   432      24363       1  (^GSPC, ^GSPC)  \
2020-01-03      0.724997  0.21   4.5   94.02696  4.0522     3234.850098   
2020-01-10      4.051150  0.21   4.5   94.02696  4.0745     3265.350098   
2020-01-17      3.963479  0.21   4.5   94.02696  4.1837     3329.620117   
2020-01-24      3.780082  0.21   4.5   94.02696  4.1769     3295.469971   
2020-01-31      3.870058  0.21   4.5   94.02696  4.2695     3225.520020   
...                  ...   ...   ...        ...     ...             ...   
2025-07-04      2.580007  0.26  15.0  113.16809  5.4090             NaN   
2025-07-11      2.015503  0.26  15.0  113.16809  5.5722     6259.750000   
2025-07-18      2.108633  0.26  15.0  113.16809  5.5466     6296.790039   
2025-07-25      1.776105  0.26  15.0  113.16809  5.5426     6388.640137   
2025-08-01      1.660700 -0.11  15.0  110.42518  5.5436             NaN   

            (^VIX, ^VIX)  (^TNX, ^TNX)  (BZ=F, BZ=F)  (BRL=X, BRL=X)  \
2020-01-03         14.02   

In [32]:
nans = Features_B.isna().sum()
print(nans[nans > 0])

(^GSPC, ^GSPC)             4
(^VIX, ^VIX)               4
(^TNX, ^TNX)               4
(BZ=F, BZ=F)               3
(BRL=X, BRL=X)             1
(IRFM11.SA, IRFM11.SA)    15
dtype: int64


In [33]:
Features_B = Features_B.ffill()
Features_B = Features_B.dropna()

print(f"Total de NaNs: {Features_B.isna().sum().sum()}")

Total de NaNs: 0


In [34]:
Features_B.columns

Features_B.columns = [
    'scoreSemanal',
    'ipcaMensal',
    'selicMeta',
    'ibcBrActivity',
    'dolarPtax',
    'sp500Index',
    'vixIndex',
    'treasuryYield10y',
    'brentOilPrice',
    'usdBrlExchange',
    'irfm11FixedIncome',
    'IPCA_Ano_Atual'
]

Features_B.columns

Index(['scoreSemanal', 'ipcaMensal', 'selicMeta', 'ibcBrActivity', 'dolarPtax',
       'sp500Index', 'vixIndex', 'treasuryYield10y', 'brentOilPrice',
       'usdBrlExchange', 'irfm11FixedIncome', 'IPCA_Ano_Atual'],
      dtype='object')

In [29]:
print(Features_B)

            scoreSemanal  ipcaMensal  selicMeta  ibcBrActivity  dolarPtax  \
2020-01-03      0.724997        0.21        4.5       94.02696     4.0522   
2020-01-10      4.051150        0.21        4.5       94.02696     4.0745   
2020-01-17      3.963479        0.21        4.5       94.02696     4.1837   
2020-01-24      3.780082        0.21        4.5       94.02696     4.1769   
2020-01-31      3.870058        0.21        4.5       94.02696     4.2695   
...                  ...         ...        ...            ...        ...   
2025-07-04      2.580007        0.26       15.0      113.16809     5.4090   
2025-07-11      2.015503        0.26       15.0      113.16809     5.5722   
2025-07-18      2.108633        0.26       15.0      113.16809     5.5466   
2025-07-25      1.776105        0.26       15.0      113.16809     5.5426   
2025-08-01      1.660700       -0.11       15.0      110.42518     5.5436   

             sp500Index  vixIndex  treasuryYield10y  brentOilPrice  \
2020-